In [1]:
! pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 2.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset, Dataset
from typing import List, Dict, Optional

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load datasets
import pandas as pd    
jsonObj = pd.read_json(path_or_buf="few_shots.json", lines=True)
fs = Dataset.from_pandas(jsonObj)

jsonObj = pd.read_json(path_or_buf="base.json", lines=True)
ds = Dataset.from_pandas(jsonObj)

In [4]:
few_shot_examples = "You're a helpful assisstant. Please answer the question after 'Answer:'\n"
for q in fs:
    few_shot_examples += f"\nQuestion: {q['query']}\n"
    few_shot_examples += f"Answer: {q['doc']}\n"


# few_shot_examples = [ {"role":"system", "content" : "You're a helpful assisstant. Please answer the question after 'Answer:'"},]
# for q in fs:  
#     few_shot_examples.append({"role":"user", "content": q['query']})
#     few_shot_examples.append({"role": "assistant", "content": q['doc']} )
print(few_shot_examples)

You're a helpful assisstant. Please answer the question after 'Answer:'

Question: why are you vegetarian?
Answer: I believe we were never meant to eat meat. We can survive quite happily and healthily without it.

Question: why am i always bloated 24/7?
Answer: Those symptoms come from bad diet and a sedentary lifestyle.  Although when they are that bad simply changing your diet will not immediately help. I strongly recommend you go to a naturopath that specializes in diet so that they can help you select a regimen of gut bacteria and vitamins that will help your digestion get back on track.. . It may also help to call local chiropractors to find a practitioner of Loomis Digestive Enzyme Therapy.

Question: What is the difference between fat free milk and non fat milk?
Answer: Fat Free Milk is from cows that are fat but the fat is removed from the milk. . Non Fat Milk is from cows that have always been fit-n-trim.  You can drink it straight from the cow

Question: How bad do you have t

In [5]:
device = "cuda"
model = AutoModelForCausalLM.from_pretrained("../models/llama2-7B-chat", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("../models/llama2-7B-chat", local_files_only=True)

model.to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.11s/it]
/opt/conda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [6]:
def truncate(text: str, delimiters: List[str]) -> str:
    for d in delimiters:
        text = text.split(d)[0]
    return text

def generate(prompt: str, max_new_tokens:int,*args, **kwargs) -> str:
    '''Generate a single response'''
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(input_ids, max_new_tokens = max_new_tokens, do_sample=True, **kwargs)
    input_length = input_ids.shape[1]
    decoded = tokenizer.batch_decode(output_ids[:, input_length:])[0]
    return decoded

import json
def to_json(example, added, filename="completions.json"):
    ex = {
        **example,
        **added
    }
    with open(filename, "a") as f:
        json.dump(ex, f)
        f.write("\n")  # 添加换行符以确保每条记录都是单独的一行
    return ex

In [7]:
ds

Dataset({
    features: ['query_id', 'doc_id', 'relevance', 'iteration', 'query', 'doc'],
    num_rows: 200
})

In [10]:
from tqdm import tqdm

def generate_responses(ds, 
                       max_new_tokens:int = 256, 
                       delimiters: List[str] = ['\n\n', tokenizer.eos_token], 
                       *args,
                       **kwargs) -> List[str]:
    '''
    Generate responses for a list of prompts. We don't use batch encode here.

    Args:
        prompts (List[str]): A list of prompts.
        max_new_tokens (int, optional): The maximum number of tokens to generate for each response. Defaults to 256.
        delimiters (List[str], optional): A list of delimiters used to truncate the generated response. Defaults to ['\n\n', tokenizer.eos_token].
        *args: Variable length argument list.
        **kwargs: Arbitrary keyword arguments.

    Returns:
        List[str]: A list of generated responses.
    '''
    responses = []
    for i in tqdm(range(len(ds))):
        prompt = few_shot_examples+f"\nQuestion: {ds[i]['query']}"
        res = generate(prompt, max_new_tokens,*args, **kwargs)
        responses.append(res)
        to_json(ds[i], {"answer": res}, 'llama2-7b-chat.json.t')
    return responses

In [11]:
responses = generate_responses(ds)
dataset = ds.add_column('responses', responses)

dataset.to_json('llama2-7b-chat.jsonl')

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 108.82ba/s]


234157